<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_train_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#제작한 세 종류의 데이터셋으로 모델을 훈련하는 코드. 훈련 결과물은 Pretrained_Model, Pretrained_Model_sentence_embed, Finetuned_Model_judge_question, Finetuned_Model_judge_answer 4종류

In [ ]:
!pip install soynlp
!pip install datasets
!pip install accelerate -U
!pip install kss
import accelerate
import random
import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
import torch.nn as nn
from kss import Kss
from transformers.modeling_outputs import SequenceClassifierOutput
from datasets import Dataset, load_dataset, ClassLabel
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer
from tqdm import tqdm
from torch.nn.functional import cross_entropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoModel, AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction,  TrainingArguments, BertForMaskedLM, Trainer, TrainerCallback, BertConfig, BertForSequenceClassification
from sklearn.model_selection import StratifiedKFold, train_test_split
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-ma

In [ ]:
#과적합 방지를 위한 EarlyStopping 클래스
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, patience=3):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def on_evaluate(self, args, state, control, **kwargs):
        # 현재 검증 손실 가져오기
        current_loss = kwargs['metrics']['eval_loss']

        # 최고 성능 갱신 또는 카운터 증가
        if self.best_score is None:
            self.best_score = current_loss
        elif current_loss > self.best_score:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                control.should_training_stop = True
        else:
            self.best_score = current_loss
            self.counter = 0

#MLM으로 비지도 학습(KcBERT, KoSimCSE_BERT)

In [ ]:
#훈련 데이터 불러오기
pretrain_dataset_path = '/content/drive/My Drive/KcBERT_pretrain_dataset_short.csv'
df_pretrain = pd.read_csv(pretrain_dataset_path, index_col = 0)
print(df_pretrain)

                                                    text
0                                                   잼나네요
1      근데 line space를 1.2 로주면 line height 이 1.2 가 그대로...
2                                       앗 뭔가 했네요 ㅋㅋㅋㅋㅋㅋㅋ
3                         긍정(신기하다, 대단하다) VS 부정(그런 거 되겠냐)
4      font-family 이름이랑 업로드하실때 넣는 Font name이랑 매치해야되는 ...
...                                                  ...
29329                               간단한 백엔드 구성이 필요해보입니다.
29330                             아니 이렇게 정성스럽게 정리해주시다니..
29331                                             이상하게..
29332                                        항상 응원하는 팬ㅎㅎ
29333                                            역시 그렇군요

[29334 rows x 1 columns]


In [ ]:
#soynlp tokenizer를 Pretrain 데이터로 학습
pretrain_list = df_pretrain['text'].to_list()

word_extractor = WordExtractor(min_frequency = 5)
word_extractor.train(pretrain_list)
word_score_table = word_extractor.extract()

training was done. used memory 1.435 Gb
all cohesion probabilities was computed. # words = 26971
all branching entropies was computed # words = 60671
all accessor variety was computed # words = 60671


In [ ]:
print(word_score_table['picker'])

Scores(cohesion_forward=0.4753297603722286, cohesion_backward=0.3333333333333333, left_branching_entropy=0, right_branching_entropy=1.4276786533747516, left_accessor_variety=0, right_accessor_variety=5, leftside_frequency=33, rightside_frequency=5)


In [ ]:
# 예제로 몇 가지 단어 점수 확인
for word, score in word_score_table.items():
    print(f"{word}: {score.cohesion_forward:.2f}")

In [ ]:
cohesion_score = {word:score.cohesion_forward for word, score in word_score_table.items()}

In [ ]:
Ltokenizer = LTokenizer(scores = cohesion_score)
Maxtokenizer = MaxScoreTokenizer(scores = cohesion_score)

In [ ]:
sentence = '자바스크립트 개발 경험이 있으신 경우라면 DateTime Picker 엘레먼트에 ID를 부여해서 자바스크립트 코드로 date 함수와 DOM을 핸들링해서 직접 처리해서 쉽게 구현하실 수가 있을 듯 합니다.'

print(Ltokenizer.tokenize(sentence))
print(Maxtokenizer.tokenize(sentence))

['자바스크립트', '개발', '경험이', '있으신', '경우', '라면', 'Dat', 'eTime', 'Picker', '엘레먼트', '에', 'ID', '를', '부여해서', '자바스크립트', '코드', '로', 'dat', 'e', '함수', '와', 'DOM을', '핸들링', '해서', '직접', '처리', '해서', '쉽게', '구현', '하실', '수가', '있을', '듯', '합니다', '.']
['자바스크립트', '개발', '경험이', '있으신', '경우', '라면', 'Dat', 'e', 'Time', 'Pic', 'ke', 'r', '엘레먼트', '에', 'ID', '를', '부여해서', '자바스크립트', '코드', '로', 'dat', 'e', '함수', '와', 'DOM을', '핸들링', '해서', '직접', '처리', '해서', '쉽게', '구현', '하실', '수가', '있을', '듯', '합니다', '.']


In [ ]:
sentence = ' 지원하는 기능이 많을수록 느립니다. BDK는 확실히 로딩속도가 느립니다.Natively는 기능이 적은 대신에 로딩속도는 조금 빠른 편이더군요. :)'

print(Ltokenizer.tokenize(sentence))
print(Maxtokenizer.tokenize(sentence))

['지원하는', '기능', '이', '많을수록', '느립니다.', 'BDK', '는', '확실히', '로딩속도가', '느립니다.', 'Natively는', '기능', '이', '적은', '대신에', '로딩속도', '는', '조금', '빠른', '편이', '더군요.', ':)']
['지', '원하는', '기능', '이', '많을수록', '느립니다.', 'BDK', '는', '확실히', '로딩', '속도', '가', '느립니다.', 'Natively', '는', '기능', '이', '적은', '대신에', '로딩', '속도', '는', '조금', '빠른', '편이', '더군요.', ':)']


In [ ]:
# 높은 cohesion 점수를 가진 단어들을 선택하여 단어사전 구성
threshold = 0.1  # 점수 기준 설정
new_vocab = [word for word, score in word_score_table.items() if score.cohesion_forward > threshold]

# 선택된 단어 출력
print(new_vocab)

['느낌', '계신', '치면', '모바', '탈퇴', '대해', '구글', 'is', '낫지', '숨겨', '맥북', '친구', '했는', '배워', '놓치', '네이', '저장', '화면', '큐레', 'Bu', '밖에', '멋집', '듣고', '툴을', '코드', '빨라', 'Sa', '티어', 'nu', '무엇', '항상', '던져', '다음', 'go', '쭉~', 'Li', '까먹', 'Vi', 'wi', 'Ty', '시간', '업로', '직접', '횟수', '툴이', '권한', '폼을', '컴퓨', '권장', '담아', '씨름', '중에', '아이', '컨디', '삼아', '플러', '쓸수', '줄여', '형식', '캐시', '싶습', 'yo', '추가', '저는', '북마', '위해', 'ja', 'OO', 'me', '여러', '복잡', '몰라', '현재', '즐거', '인풋', '30', '쉽지', 'El', '최소', 'Wo', '목록', '코딩', '깨닫', '캠페', '팀원', '규칙', '민호', '됐다', '클릭', '표시', '텍스', '쌓인', '40', '뒷풀', '돈을', 'ne', 'mi', '송금', '양해', '깊게', '휴지', '숨기', '증상', '광고', 'ho', 'fr', '빈값', '패스', '벌어', '겪고', '검색', '방법', '메일', '상세', '듯합', '끌고', '재은', '<-', '오픈', '깨지', '끝내', '국내', '급한', '답변', '역시', '되는', '챗G', '갠톡', '넵!', '갔다', '??', 'Ex', '실제', '—>', '즐겨', '대한', '※버', 'mo', 'La', '흠흠', '봤을', '데이', '배포', 'UR', '최근', '>=', '돌아', '낮더', '60', '버튼', '뜨는', '단위', '분들', '라고', '훨씬', '같은', '라는', '급하', '눌러', '파이', '하나', '싱크', '휴식', '효과', '호출', '/>', '올라

In [ ]:
#MLM을 위한 데이터셋의 일부를 [MASK] 토큰으로 교체하는 함수
def mask_input_ids(input_ids, tokenizer, mlm_probability=0.15):
    labels = input_ids.clone()

    #각 입력에서 특수 토큰은 마스킹하지 않도록 설정
    special_tokens_mask = [
      torch.tensor(tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True), dtype = torch.bool)
      for val in input_ids]
    special_tokens_mask = torch.stack(special_tokens_mask)

    #마스킹 확률을 적용해 마스크 생성
    probability_matrix = torch.full(labels.shape, mlm_probability)
    probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()

    # PyTorch에서 사용하는 masked loss 계산을 위해 마스킹 되지 않은 부분은 -100으로 설정
    labels[~masked_indices] = -100

    # `[MASK]` 토큰으로 대체
    input_ids[masked_indices] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)
    return input_ids, labels

#KcBERT 훈련

In [ ]:
#Huggingface의 원본 모델 로드
#BertForMaskedLM 구조에 맞게 모델 불러오기
HUGGINGFACE_MODEL_PATH = 'beomi/kcbert-base'

KcBERT_tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
KcBERT_model = BertForMaskedLM.from_pretrained(HUGGINGFACE_MODEL_PATH)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Trainer가 알아서 gpu를 감지하고 모델과 데이터를 gpu로 옮겨줌
#GPU 사용 코드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#파이프라인을 사용하는 경우 GPU로 모델을 옮겨준 뒤 파이프라인에 모델을 넣어줘야 함
KcBERT_model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
# 새로운 단어 토크나이저에 추가
num_added_toks = KcBERT_tokenizer.add_tokens(new_vocab)
print(f"Added {num_added_toks} tokens")

# 모델의 임베딩 크기 조정
KcBERT_model.resize_token_embeddings(len(KcBERT_tokenizer))

Added 16829 tokens


Embedding(44857, 768)

In [ ]:
#MLM 데이터셋 준비
corpus = pretrain_list
tokenized_corpus = KcBERT_tokenizer(corpus, add_special_tokens = True, max_length = KcBERT_model.config.max_position_embeddings, truncation = True, padding = 'max_length', return_tensors = 'pt')

In [ ]:
print(tokenized_corpus)

{'input_ids': tensor([[    2, 10306,  9206,  ...,     0,     0,     0],
        [    2, 18702,  1572,  ...,     0,     0,     0],
        [    2, 39402,  8606,  ...,     0,     0,     0],
        ...,
        [    2, 39950,    17,  ...,     0,     0,     0],
        [    2,  8108, 35185,  ...,     0,     0,     0],
        [    2, 29952, 10220,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
#MLM을 위한 masked 데이터셋 생성(MLM에서는 stratify 적용하지 않음.)
masked_inputs, labels = mask_input_ids(tokenized_corpus['input_ids'], KcBERT_tokenizer)
pretrain_dict = {'input_ids' : masked_inputs, 'attention_mask' : tokenized_corpus['attention_mask'], 'labels' : labels}
pretrain_dataset = Dataset.from_dict(pretrain_dict)
pretrain_dataset = pretrain_dataset.train_test_split(test_size = 0.1)

In [ ]:
# 훈련 설정
training_args = TrainingArguments(
    output_dir = './results',
    evaluation_strategy = 'steps',
    eval_steps = 500,
    save_strategy = "steps",
    save_steps = 500,
    num_train_epochs = 3,
    save_total_limit = 3,
    per_device_eval_batch_size = 8,
    per_device_train_batch_size = 8,
    warmup_steps = 300, #고려사항
    weight_decay = 0.01, #고려사항
    logging_dir = "./logs",
    load_best_model_at_end = True
)

# 트레이너 생성
trainer = Trainer(
    model = KcBERT_model,
    args = training_args,
    train_dataset = pretrain_dataset['train'],
    eval_dataset = pretrain_dataset['test'],
    callbacks = [EarlyStoppingCallback(patience = 5)]
)

# 훈련 시작
trainer.train()

Step,Training Loss,Validation Loss
500,7.463500,7.170615
1000,7.100200,6.953511
1500,6.873800,6.824831
2000,6.812900,6.713923


In [ ]:
 #google drive에 학습된 모델을 저장
save_path = '/content/drive/My Drive/Pretrained_Model'

KcBERT_model.save_pretrained(save_path)
KcBERT_tokenizer.save_pretrained(save_path)

#KoSimCSE_BERT 훈련

In [ ]:
#먼저 pooler 층을 변수로 저장해두기 위해 모델을 AutoModel로 로드
HUGGINGFACE_MODEL_PATH = 'BM-K/KoSimCSE-bert-multitask'

KoSim_model = AutoModel.from_pretrained(HUGGINGFACE_MODEL_PATH)

#pooler 층의 파라미터만 변수로 저장
pooler_layer = KoSim_model.pooler

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [ ]:
#MLM으로 domain adaptation을 진행하기 위해 모델을 BertForMaskedLM으로 로드
HUGGINGFACE_MODEL_PATH = 'BM-K/KoSimCSE-bert-multitask'

KoSim_model = BertForMaskedLM.from_pretrained(HUGGINGFACE_MODEL_PATH)
KoSim_tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
KoSim_model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
print(KoSim_model.pooler.dense.weight)

Parameter containing:
tensor([[ 3.1855e-02, -5.4072e-03, -4.4124e-02,  ...,  2.3173e-02,
          2.5717e-02,  2.5337e-02],
        [ 2.8142e-02, -8.6567e-03,  2.3887e-02,  ...,  1.1529e-02,
         -1.3463e-02, -1.9435e-03],
        [ 1.3164e-02,  5.6479e-03, -2.9252e-02,  ..., -2.5653e-02,
          7.2856e-03, -2.1704e-02],
        ...,
        [ 2.3056e-03,  6.7739e-03,  1.0147e-03,  ..., -1.3549e-02,
         -1.3426e-03,  3.1372e-02],
        [ 7.8224e-03, -8.6145e-03,  3.3083e-02,  ...,  3.2418e-02,
         -3.4644e-07, -2.8555e-02],
        [-3.0232e-02,  1.0719e-02,  2.0621e-02,  ...,  4.9263e-02,
         -5.2133e-02,  6.1498e-02]], requires_grad=True)


In [ ]:
# 새로운 단어 토크나이저에 추가
num_added_toks = KoSim_tokenizer.add_tokens(new_vocab)
print(f"Added {num_added_toks} tokens")

# 모델의 임베딩 크기 조정
KoSim_model.resize_token_embeddings(len(KoSim_tokenizer))

Added 12604 tokens


Embedding(43041, 768)

In [ ]:
#MLM 데이터셋 준비
corpus = pretrain_list
tokenized_corpus = KoSim_tokenizer(corpus, add_special_tokens = True, max_length = KoSim_model.config.max_position_embeddings, truncation = True, padding = 'max_length', return_tensors = 'pt')

In [ ]:
print(tokenized_corpus)

{'input_ids': tensor([[    2, 10306,  9206,  ...,     0,     0,     0],
        [    2, 18702,  1572,  ...,     0,     0,     0],
        [    2, 39402,  8606,  ...,     0,     0,     0],
        ...,
        [    2, 39950,    17,  ...,     0,     0,     0],
        [    2,  8108, 35185,  ...,     0,     0,     0],
        [    2, 29952, 10220,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
#MLM을 위한 masked 데이터셋 생성(MLM에서는 stratify 적용하지 않음.)
masked_inputs, labels = mask_input_ids(tokenized_corpus['input_ids'], KoSim_tokenizer)
pretrain_dict = {'input_ids' : masked_inputs, 'attention_mask' : tokenized_corpus['attention_mask'], 'labels' : labels}
pretrain_dataset = Dataset.from_dict(pretrain_dict)
pretrain_dataset = pretrain_dataset.train_test_split(test_size = 0.1)

In [ ]:
# 훈련 설정
training_args = TrainingArguments(
    output_dir = './results',
    evaluation_strategy = 'steps',
    eval_steps = 500,
    save_strategy = "steps",
    save_steps = 500,
    num_train_epochs = 3,
    save_total_limit = 3,
    per_device_eval_batch_size = 8,
    per_device_train_batch_size = 8,
    warmup_steps = 300, #고려사항
    weight_decay = 0.01, #고려사항
    logging_dir = "./logs",
    load_best_model_at_end = True
)

# 트레이너 생성
trainer = Trainer(
    model = KoSim_model,
    args = training_args,
    train_dataset = pretrain_dataset['train'],
    eval_dataset = pretrain_dataset['test'],
    callbacks = [EarlyStoppingCallback(patience = 5)]
)

# 훈련 시작
trainer.train()

Step,Training Loss,Validation Loss
500,8.810600,8.098687
1000,7.728400,7.515493
1500,7.425400,7.267002
2000,7.166300,7.091899
2500,6.963800,6.929874
3000,6.897800,6.813987
3500,6.437900,6.691690
4000,5.818000,6.640306
4500,5.842000,6.571889
5000,5.867900,6.499487


KeyboardInterrupt: 

In [ ]:
#학습이 끝난 후 MLM을 위한 cls층을 제거하고, 변수로 저장해둔 pooler 층을 다시 추가
del KoSim_model.cls
KoSim_model.bert.pooler = pooler_layer
print(KoSim_model)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(43041, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [ ]:
#google drive에 학습된 모델을 저장
save_path = '/content/drive/My Drive/Pretrained_Model_sentence_embed'

KoSim_model.save_pretrained(save_path)
KoSim_tokenizer.save_pretrained(save_path)

('/content/drive/My Drive/Pretrained_Model_sentence_embed/tokenizer_config.json',
 '/content/drive/My Drive/Pretrained_Model_sentence_embed/special_tokens_map.json',
 '/content/drive/My Drive/Pretrained_Model_sentence_embed/vocab.txt',
 '/content/drive/My Drive/Pretrained_Model_sentence_embed/added_tokens.json',
 '/content/drive/My Drive/Pretrained_Model_sentence_embed/tokenizer.json')

#질문 데이터셋으로 파인튜닝

In [ ]:
#훈련 데이터 불러오기
finetune_dataset_path = '/content/drive/My Drive/KcBERT_finetune_dataset_question.xlsx'
df_finetune = pd.read_excel(finetune_dataset_path, index_col = 0)

In [ ]:
#모델 불러오기
pretrained_model_path = '/content/drive/My Drive/Pretrained_Model'

pretrained_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path)
pretrained_model = BertForSequenceClassification.from_pretrained(pretrained_model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/My Drive/Pretrained_Model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Trainer가 알아서 gpu를 감지하고 모델과 데이터를 gpu로 옮겨줌
#GPU 사용 코드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#파이프라인을 사용하는 경우 GPU로 모델을 옮겨준 뒤 파이프라인에 모델을 넣어줘야 함
pretrained_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(40596, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
#훈련 데이터셋의 결측치 제거
print(df_finetune[pd.isna(df_finetune['text']) == True])
df_finetune = df_finetune[pd.isna(df_finetune['text']) == False]
print(df_finetune)

    text  label
814  NaN      0
815  NaN      0
816  NaN      0
817  NaN      0
818  NaN      0
..   ...    ...
991  NaN      0
992  NaN      0
993  NaN      0
994  NaN      0
995  NaN      0

[182 rows x 2 columns]
                                                   text  label
0     안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...      1
1     안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...      1
2     안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...      1
3                                  API키는 분리해서 쓰시면 좋겠습니다      1
4                                                     .      1
...                                                 ...    ...
3584   안녕하세요. 왕초보자입니다. 리피팅 그룹을 따라 만들어보고 있는데 저는 스크롤에 ...      1
3585                                 영상의 에디터가 옛날버전인것같네요      0
3586   그래서 저는 이것저것에서 스크롤 기능을찾아보는데 없네요... 업데이트 되면서 리피...      1
3587   지금은 1*4로. 리피팅 그룹이 고정된 상태시네요 가로 스크롤을 구현하려고 하시는...      0
3588             그럼 row값을 1로 고정하시고 column은 Fixed를 푸셔야해요      0

[3407 rows x 2 columns]


In [ ]:
#파인튜닝을 위한 데이터셋 준비, 훈련/검증 9:1 분리
def tokenize_function(examples):
    return pretrained_tokenizer(examples['text'], truncation = True, padding = "max_length", max_length = pretrained_model.config.max_position_embeddings)

dataset = Dataset.from_pandas(df_finetune)
finetune_dataset = dataset.map(tokenize_function, batched=True)
finetune_dataset = finetune_dataset.class_encode_column('label')
finetune_dataset = finetune_dataset.train_test_split(test_size = 0.1, stratify_by_column = 'label')

Map:   0%|          | 0/3407 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/3407 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/3407 [00:00<?, ? examples/s]

In [ ]:
#과적합 방지를 위한 EarlyStopping 클래스
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, patience=3):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def on_evaluate(self, args, state, control, **kwargs):
        # 현재 검증 손실 가져오기
        current_loss = kwargs['metrics']['eval_loss']

        # 최고 성능 갱신 또는 카운터 증가
        if self.best_score is None:
            self.best_score = current_loss
        elif current_loss > self.best_score:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                control.should_training_stop = True
        else:
            self.best_score = current_loss
            self.counter = 0

In [ ]:
#훈련 설정
training_args = TrainingArguments(
    output_dir = './results',
    learning_rate = 5e-5,
    evaluation_strategy = 'steps',
    eval_steps = 100,
    save_strategy = "steps",
    save_steps = 100,
    num_train_epochs = 3,
    save_total_limit = 3,
    per_device_eval_batch_size = 8,
    per_device_train_batch_size = 8,
    warmup_steps = 100, #고려사항
    weight_decay = 0.01, #고려사항
    logging_dir = "./logs",
    load_best_model_at_end = True
)

# 트레이너 생성
trainer = Trainer(
    model = pretrained_model,
    args = training_args,
    train_dataset = finetune_dataset['train'],
    eval_dataset = finetune_dataset['test'],
    callbacks = [EarlyStoppingCallback(patience = 3)]
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
100,No log,0.717251
200,No log,0.391957
300,No log,0.401136
400,No log,0.424649
500,0.270300,0.440664


TrainOutput(global_step=500, training_loss=0.2702892150878906, metrics={'train_runtime': 322.4451, 'train_samples_per_second': 28.526, 'train_steps_per_second': 3.573, 'total_flos': 615741536196000.0, 'train_loss': 0.2702892150878906, 'epoch': 1.3020833333333333})

In [ ]:
#google drive에 학습된 모델을 저장
save_path = '/content/drive/My Drive/Finetuned_Model_judge_question'

pretrained_model.save_pretrained(save_path)
pretrained_tokenizer.save_pretrained(save_path)

('/content/drive/My Drive/Finetuned_Model_judge_question/tokenizer_config.json',
 '/content/drive/My Drive/Finetuned_Model_judge_question/special_tokens_map.json',
 '/content/drive/My Drive/Finetuned_Model_judge_question/vocab.txt',
 '/content/drive/My Drive/Finetuned_Model_judge_question/added_tokens.json',
 '/content/drive/My Drive/Finetuned_Model_judge_question/tokenizer.json')

#Labeled NSP 데이터셋으로 파인튜닝

In [ ]:
#훈련 데이터 불러오기
finetune_dataset_path = '/content/drive/My Drive/KcBERT_finetune_dataset_answer.xlsx'
df_finetune = pd.read_excel(finetune_dataset_path, index_col = 0)

In [ ]:
#MLM으로 훈련된 모델 로드
#BertForNextSentencePrediction 구조에 맞게 모델 불러오기
pretrained_model_load_path = '/content/drive/My Drive/Pretrained_Model'

pretrained_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_load_path)
pretrained_model = BertForNextSentencePrediction.from_pretrained(pretrained_model_load_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of BertForNextSentencePrediction were not initialized from the model checkpoint at /content/drive/My Drive/Pretrained_Model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrained_model.to(device)

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(40596, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
print(df_finetune)
dataset = Dataset.from_pandas(df_finetune)
print(dataset)
print(df_finetune[df_finetune['sent2'].isnull()])

                                                  sent1  \
0     안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...   
1     안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
2     안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
3     안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...   
4     모두의노코드 만드신 분들께 질문 드리고 싶어요.제가 버블로 만든 서비스 관련해 모바...   
...                                                 ...   
5819   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
5820   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
5821   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
5822   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
5823   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   

                                                  sent2  label  sent2_index  
0     저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...      1          0.0  
1     헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요...      1          1.0  
2     모두의노코드를 여쭤보신 것인지 모르겠어서 이게 맞는지는 모르겠습니다.모두의노코드의 ...  

In [ ]:
# NSP 데이터셋 준비(NSP 태스크에서는 stratify 적용)
def tokenize_function(examples):
    return pretrained_tokenizer(examples['sent1'], examples['sent2'], truncation = True, padding = "max_length", max_length = pretrained_model.config.max_position_embeddings)

dataset = Dataset.from_pandas(df_finetune)
finetune_dataset = dataset.map(tokenize_function, batched=True)
finetune_dataset = finetune_dataset.class_encode_column('label')
finetune_dataset = finetune_dataset.train_test_split(test_size = 0.1, stratify_by_column = 'label')

Map:   0%|          | 0/5824 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/5824 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/5824 [00:00<?, ? examples/s]

In [ ]:
#훈련 설정
training_args = TrainingArguments(
    output_dir = './results',
    learning_rate = 2e-5, #pretrain에 비해 낮은 값
    evaluation_strategy = 'steps',
    eval_steps = 200,
    save_strategy = "steps",
    save_steps = 200,
    num_train_epochs = 3,
    save_total_limit = 3,
    per_device_eval_batch_size = 8,
    per_device_train_batch_size = 8,
    warmup_steps = 200, #고려사항
    weight_decay = 0.01, #고려사항
    logging_dir = "./logs",
    load_best_model_at_end = True
)

# 트레이너 생성
trainer = Trainer(
    model = pretrained_model,
    args = training_args,
    train_dataset = finetune_dataset['train'],
    eval_dataset = finetune_dataset['test'],
    callbacks = [EarlyStoppingCallback(patience = 3)]
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
200,No log,0.489896
400,No log,0.593015
600,0.282600,0.410021
800,0.282600,0.418134
1000,0.285700,0.466792
1200,0.285700,0.502589


TrainOutput(global_step=1200, training_loss=0.2791646353403727, metrics={'train_runtime': 708.8664, 'train_samples_per_second': 22.18, 'train_steps_per_second': 2.776, 'total_flos': 1478920519962000.0, 'train_loss': 0.2791646353403727, 'epoch': 1.8292682926829267})

In [ ]:
#google drive에 학습된 모델을 저장
save_path = '/content/drive/My Drive/Finetuned_Model_judge_answer'

pretrained_model.save_pretrained(save_path)
pretrained_tokenizer.save_pretrained(save_path)

('/content/drive/My Drive/Finetuned_Model_judge_answer/tokenizer_config.json',
 '/content/drive/My Drive/Finetuned_Model_judge_answer/special_tokens_map.json',
 '/content/drive/My Drive/Finetuned_Model_judge_answer/vocab.txt',
 '/content/drive/My Drive/Finetuned_Model_judge_answer/added_tokens.json',
 '/content/drive/My Drive/Finetuned_Model_judge_answer/tokenizer.json')